##### Copyright 2020 The TensorFlow Authors. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.



Authors : [jaeyoo@](https://github.com/jaeyoo), [khanhlvg@](https://github.com/khanhlvg), [abattery@](https://github.com/abattery), [thaink@](https://github.com/thaink) (Google Research)

Created : 2020-07-03 KST

Last updated : 2020-07-04 KST

-----
Change logs
* 2020-07-04 KST : Update notebook with the latest repo.
 * https://github.com/TensorSpeech/TensorflowTTS/pull/84 merged.
* 2020-07-03 KST : First implementation (outputs : `fastspeech_quant.tflite`)
 * varied-length input tensor, varied-length output tensor
 * Inference on tflite works well.

-----

**Status** : successfully converted (`fastspeech_quant.tflite`)

**Disclaimer** 
- This colab doesn't care about the latency, so it compressed the model with quantization. (112 MB -> 28 MB)
- The TFLite file doesn't have LJSpeechProcessor. So you need to run it before feeding input vectors.
- `tf-nightly>=2.4.0-dev20200630`


# Generate voice with FastSpeech

In [ ]:
# !pip install tf-nightly

In [1]:
import numpy as np
import soundfile as sf
import yaml
import tensorflow as tf

from tensorflow_tts.processor.kss import KSSProcessor
from tensorflow_tts.processor.kss import symbols, _symbol_to_id

from tensorflow_tts.configs import FastSpeechConfig, FastSpeech2Config
from tensorflow_tts.configs import MelGANGeneratorConfig

from tensorflow_tts.models import TFFastSpeech, TFFastSpeech2
from tensorflow_tts.models import TFMelGANGenerator

from IPython.display import Audio
print(tf.__version__) # check if >= 2.4.0

/home/lap13548/anaconda3/envs/tflite/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:44: UserWarning: You are currently using a nightly version of TensorFlow (2.4.0-dev20200708). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


2.4.0-dev20200708


In [ ]:
# initialize melgan model
with open('./examples/melgan/conf/melgan.v1.yaml') as f:
    melgan_config = yaml.load(f, Loader=yaml.Loader)
melgan_config = MelGANGeneratorConfig(**melgan_config["generator_params"])
melgan = TFMelGANGenerator(config=melgan_config, name='melgan_generator')
melgan._build()
melgan.load_weights("./examples/melgan/pretrained/generator-1500000.h5")

In [2]:
# initialize FastSpeech model.
with open('../examples/fastspeech2/conf/fastspeech2.v1.yaml') as f:
    config = yaml.load(f, Loader=yaml.Loader)
config = FastSpeech2Config(**config["fastspeech_params"])
fastspeech = TFFastSpeech2(config=config, name="fastspeech2",
                          enable_tflite_convertible=True)

# Newly added :
fastspeech._build()
fastspeech.load_weights("../examples/fastspeech2/checkpoints/model-150000.h5")

In [ ]:
input_text = "Recent research at Harvard has shown meditating\
for as little as 8 weeks, can actually increase the grey matter in the \
parts of the brain responsible for emotional regulation, and learning."

processor = KSSProcessor(None, "korean_cleaners")
input_ids = processor.text_to_sequence(input_text.lower())
input_ids = np.concatenate([input_ids, [len(symbols) - 1]], -1)  # eos.

mel_before, mel_after, duration_outputs, _, _ = fastspeech.inference(
    input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
    attention_mask=tf.math.not_equal(tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0), 0),
    speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
    speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    f0_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    energy_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
)

# audio_before = melgan(mel_before)[0, :, 0]
# audio_after = melgan(mel_after)[0, :, 0]

In [ ]:
# Audio(data=audio_before, rate=22050)

In [ ]:
# Audio(data=audio_after, rate=22050)

# Convert to TFLite

In [ ]:
# Concrete Function
fastspeech_concrete_function = fastspeech.inference_tflite.get_concrete_function()

In [ ]:
converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [fastspeech_concrete_function]
)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

In [ ]:
# Save the TF Lite model.
with open('fastspeech_quant.tflite', 'wb') as f:
  f.write(tflite_model)

print('Model size is %f MBs.' % (len(tflite_model) / 1024 / 1024.0) )

In [ ]:
## Download the TF Lite model
#from google.colab import files
#files.download('fastspeech_quant.tflite') 

# Inference from TFLite

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path='fastspeech_quant.tflite')

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare input data.
def prepare_input(input_ids):
  input_ids = tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0)
  return (input_ids,
          tf.math.not_equal(input_ids, 0),
          tf.convert_to_tensor([0], tf.int32),
          tf.convert_to_tensor([1.0], dtype=tf.float32))

# Test the model on random input data.
def infer(input_text):
  for x in input_details:
    print(x)
  for x in output_details:
    print(x)
  processor = LJSpeechProcessor(None, "english_cleaners")
  input_ids = processor.text_to_sequence(input_text.lower())
  interpreter.resize_tensor_input(input_details[0]['index'], 
                                  [1, len(input_ids)])
  interpreter.resize_tensor_input(input_details[1]['index'], 
                                  [1, len(input_ids)])
  interpreter.resize_tensor_input(input_details[2]['index'], 
                                  [1])
  interpreter.resize_tensor_input(input_details[3]['index'], 
                                  [1])
  interpreter.allocate_tensors()
  input_data = prepare_input(input_ids)
  for i, detail in enumerate(input_details):
    input_shape = detail['shape']
    interpreter.set_tensor(detail['index'], input_data[i])

  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  return (interpreter.get_tensor(output_details[0]['index']),
          interpreter.get_tensor(output_details[1]['index']))

In [ ]:
input_text = "Recent research at Harvard has shown meditating\
for as little as 8 weeks, can actually increase the grey matter in the \
parts of the brain responsible for emotional regulation, and learning."

decoder_output_tflite, mel_output_tflite = infer(input_text)
audio_before_tflite = melgan(decoder_output_tflite)[0, :, 0]
audio_after_tflite = melgan(mel_output_tflite)[0, :, 0]

In [ ]:
Audio(data=audio_before_tflite, rate=22050)

In [ ]:
Audio(data=audio_after_tflite, rate=22050)

In [ ]:
input_text = "I love TensorFlow Lite converted FastSpeech with quantization. \
The converted model file is of 28.6 Mega bytes."

decoder_output_tflite, mel_output_tflite = infer(input_text)
audio_before_tflite = melgan(decoder_output_tflite)[0, :, 0]
audio_after_tflite = melgan(mel_output_tflite)[0, :, 0]

In [ ]:
Audio(data=audio_before_tflite, rate=22050)

In [ ]:
Audio(data=audio_after_tflite, rate=22050)